[View in Colaboratory](https://colab.research.google.com/github/sohamshashank1/MLtoolkit/blob/master/indix.ipynb)

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

'/device:GPU:0'

In [2]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1tKuZW05YpMnsLGYTwVJ10rICe8SxTD7B'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
test_downloaded = drive.CreateFile({'id': '1VBwFrY_Rj5MPsnhK0VODcTzl4SVZN8BZ'})
test_downloaded.GetContentFile('bmv_test_set.csv') 

In [0]:
train_downloaded = drive.CreateFile({'id': '1tKuZW05YpMnsLGYTwVJ10rICe8SxTD7B'})
train_downloaded.GetContentFile('bmv_training_set.csv')

In [0]:
bigram_downloaded = drive.CreateFile({'id': '1ujfJloacuA5j8zFEF8mhFEHyloXAS6g4'})
bigram_downloaded.GetContentFile('bigrams.csv')

In [0]:
import pandas as pd

In [0]:
tr = pd.read_csv('bmv_training_set.csv', sep = ',',  encoding='latin1')

In [0]:
ts = pd.read_csv('bmv_test_set.csv', sep = ',',  encoding='latin1')

In [28]:
bigrams = pd.read_csv('bigrams.csv', sep = ',',  encoding='latin1')
bigrams['bigram'] = bigrams['bigram'].apply(lambda x: x.lower())
bigrams

,bigram
0,a b c d e f g h i j k l m n o p q r s t u v...


In [29]:
tr.head()

,id,additionalAttributes,label
0,0,Store Item Number (DPCI)=244-02-6685;Pop Music...,music
1,1,Package Dimensions=13.8 x 13 x 3.2 inches;Best...,rest
2,2,color=GREY;size=medium;Sizing:=Junior,rest
3,3,Product Dimensions=12 x 0 x 6 inches;ASIN=B00C...,rest
4,4,size=7,rest


In [0]:
tr.columns = ['id','att', 'label']
ts.columns = ['id','att']
tr['att'] = tr['att'].str.replace('[^a-zA-Z\s]',' ')
ts['att'] = ts['att'].str.replace('[^a-zA-Z\s]',' ')
tr['att'] = tr['att'].apply(lambda x: x.lower())
ts['att'] = ts['att'].apply(lambda x: x.lower())

In [32]:
ts.head()

,id,att
0,0,movie genre sports recreation store item num...
1,1,author francis ford coppola featuring marlon b...
2,2,performer phil hamilton record label select o ...
3,7,color chrome
4,9,format hardcover


In [0]:
import re
tr['att'] = tr['att'].apply(lambda x: x.strip())
tr['att'] = tr['att'].apply(lambda x: re.sub(r'\s+', ' ', x))
ts['att'] = ts['att'].apply(lambda x: x.strip())
ts['att'] = ts['att'].apply(lambda x: re.sub(r'\s+', ' ', x))

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
bigram_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2), token_pattern=r'\b\w+\b', min_df=1)
countvec1 = CountVectorizer()
# countvec2 = CountVectorizer()
big_dtm = bigram_vectorizer.fit_transform(bigrams.bigram)
big_dtm

<1x729 sparse matrix of type '<class 'numpy.int64'>'
	with 729 stored elements in Compressed Sparse Row format>

In [35]:
train_dtm = bigram_vectorizer.transform(tr.att)
test_dtm = bigram_vectorizer.transform(ts.att)
train_dtm

<300000x729 sparse matrix of type '<class 'numpy.int64'>'
	with 29507761 stored elements in Compressed Sparse Row format>

In [36]:
test_dtm

<60000x729 sparse matrix of type '<class 'numpy.int64'>'
	with 5852043 stored elements in Compressed Sparse Row format>

In [37]:
len(ts)

60000

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, n_estimators= 20)
rf_model = rf.fit(train_dtm, tr['label'])


In [39]:
rf_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=70, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [0]:
predicted_df = pd.DataFrame(rf_model.predict(test_dtm))

In [21]:
uploaded = drive.CreateFile({'predicted_df': 'pred_df.csv'})
uploaded.SetContentFile('pred_df.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 10vn7DxecYOD2v7bYYNclOQkE4qqn29xL


In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(tr['label'] == predicted_df)

In [0]:
from sklearn import metrics
scores = cross_val_score(rf_model, train_dtm, tr['label'], cv=5, scoring='f1_macro')
scores